In [ ]:
import pandas as pd
import re
import string
from transformers import pipeline
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [7]:
data = pd.read_excel("Review KCU Pulau Jawa.xlsx")

In [8]:
data = pd.read_excel("Review KCU Pulau Jawa.xlsx")
data = data.dropna(subset=['review_text'])

data['review_text_cleaned'] = data['review_text'].astype(str).str.lower()
data['review_text_cleaned'] = [re.sub(r'[^\x00-\x7f]',r'', i) for i in data['review_text_cleaned']]
data['review_text_cleaned'] = [re.sub(r'\n', r' ', i) for i in data['review_text_cleaned']]
data['review_text_cleaned'] = data['review_text_cleaned'].apply(
    lambda x: re.sub(f"[{re.escape(string.punctuation)}]", " ", x)
)
# Remove numbers
data['review_text_cleaned'] = data['review_text_cleaned'].apply(lambda x: re.sub(r'\d+', '', x))

# Remove English words (basic filtering using a set of common English words)
nltk.download('words')
english_words = set(words.words())

def remove_english_words(text):
    return ' '.join([word for word in text.split() if word not in english_words])

data['review_text_cleaned'] = data['review_text_cleaned'].apply(remove_english_words)

# Normalize whitespace again after filtering
data['review_text_cleaned'] = data['review_text_cleaned'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Drop empty rows again
data = data[data['review_text_cleaned'] != ""]

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\POS\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [12]:
from collections import Counter

all_words = ' '.join(data['review_text_cleaned']).split()
word_freq = Counter(all_words)

In [14]:
import re

def has_repeated_letters(word):
    return re.search(r'(.)\1{2,}', word) is not None

suspect_words = [
    w for w in word_freq
    if word_freq[w] < 5 or has_repeated_letters(w)
]
suspect_words

['cwus',
 'displaying',
 'parcels',
 'features',
 'december',
 'attempted',
 'memangnya',
 'sebatas',
 'tuliskan',
 'ditempelkan',
 'tolonglah',
 'cb',
 'jpg',
 'jakpus',
 'sminggu',
 'tlng',
 'darri',
 'sekaliini',
 'borok',
 'hoak',
 'sekalinya',
 'dijawb',
 'catatan',
 'terterah',
 'bunyi',
 'gedoran',
 'pendapat',
 'menjd',
 'setannya',
 'ketularan',
 'sangaaattt',
 'sangaat',
 'kecewaaa',
 'ngmong',
 'sebutkan',
 'knpa',
 'skarang',
 'brang',
 'kmna',
 'brsangkutan',
 'brsikap',
 'bodo',
 'cn',
 'ktsh',
 'idjkta',
 'jakartasoekarnohatta',
 'reaksinya',
 'tindaklanjutin',
 'pemeriksaan',
 'blg',
 'tersangkut',
 'dibayarkan',
 'bensin',
 'tugasnya',
 'ind',
 'customernya',
 'menanggapinya',
 'sebab',
 'kepentingan',
 'meyakinkan',
 'terusan',
 'nlpn',
 'kejauhan',
 'musim',
 'ujan',
 'yos',
 'sudarso',
 'aturannya',
 'yng',
 'lntai',
 'kapanpun',
 'pemerintag',
 'sebagus',
 'reformasi',
 'membenci',
 'mamah',
 'pki',
 'thailand',
 'lmbat',
 'dijakarta',
 'managemen',
 'fakta',
 'diu

In [ ]:
words_dict = {
    'tdk'  : 'tidak',
    'yg' : 'yang',
    'ga' : 'tidak',
    'gak' : 'tidak',
    'tp' : 'tapi',
    'd' : 'di',
    'sy' : 'saya',
    '&' : 'dan',
    'dgn' : 'dengan', 
    'utk' : 'untuk',
    'gk' : 'tidak',
    'jd' : 'jadi',
    'jg' : 'juga',
    'dr' : 'dari',
    'krn' : 'karena',
    'aja' : 'saja',
    'karna' : 'karena',
    'udah' : 'sudah',
    'kmr' : 'kamar',
    'g' : 'tidak',
    'dpt' : 'dapat',
    'banget' : 'sekali',
    'bgt' : 'sekali',
    'kalo' : 'kalau',
    'n' : 'dan', 
    'bs' : 'bisa',
    'oke' : 'ok',
    'dg' : 'dengan',
    'pake' : 'pakai',
    'sampe' : 'sampai',
    'dapet' : 'dapat',
    'ad' : 'ada',
    'lg' : 'lagi',
    'bikin' : 'buat',
    'tak' : 'tidak',
    'ny' : 'nya',
    'ngga' : 'tidak',
    'nunggu' : 'tunggu',
    'klo' : 'kalau',
    'blm' : 'belum',
    'trus' : 'terus',
    'kayak' : 'seperti',
    'dlm' : 'dalam',
    'udh' : 'sudah',
    'tau' : 'tahu',
    'org' : 'orang',
    'hrs' : 'harus',
    'msh' : 'masih',
    'sm' : 'sama',
    'byk' : 'banyak',
    'krg' : 'kurang',
    'kmar' : 'kamar',
    'spt' : 'seperti',
    'pdhl' : 'padahal',
    'chek' : 'cek',
    ' dteng2' : 'datang-datang',
    'pesen' : 'pesan', 
    'kran' : 'keran',
    'gitu' : 'begitu',
    'tpi' : 'tapi',
    'lbh' : 'lebih',
    'tmpt' : 'tempat',
    'dikasi' : 'dikasih',
    'serem' : 'seram', 
    'sya' : 'saya',
    'jgn' : 'jangan',
    'dri' : 'dari',
    'dtg' : 'datang',
    'gada' : 'tidak ada',
    'standart' : 'standar',
    'mlm' : 'malam',
    'k'  : 'ke',
    'kl' : 'kalau',
    'sgt': 'sangat',
    'y' : 'ya',
    'krna' : 'karena',
    'tgl' : 'tanggal', 
    'terimakasih' : 'terima kasih',
    'kecoak' : 'kecoa',
    'pd' : 'pada',
    'tdr' : 'tidur', 
    'jdi' : 'jadi',
    'kyk' : 'seperti',
    'sdh' : 'sudah',
    'ama' : 'sama',
    'gmana' : 'bagaimana',
    'dalem' : 'dalam',
    'tanyak' : 'tanya',
    'taru' : 'taruh',
    'gede' : 'besar',
    'kaya' : 'seperti',
    'access' : 'akses',
    'tetep' : 'tetap',
    'mgkin' : 'mungkin',
    'sower' : 'shower',
    'idup' : 'hidup',
    'nyaaa' : 'nya',
    'baikk' : 'baik',
    'hanay' : 'hanya',
    'tlp' : 'telpon',
    'kluarga' : 'keluarga',
    'jln' : 'jalan',
    'hr' : 'hari',
    'ngak' : 'tidak',
    'bli' : 'beli',
    'kmar' : 'kamar',
    'naro' : 'taruh',
    'telpon':'telepon',
    'nlpn':'telepon',
    'tlp':'telepon',
    'nelpon':'telepon',
    'ktr':'kantor',
    'kntr':'kantor',
    'cb' : 'coba',
    'bgs' : 'bagus',
    'mntp' : 'mantap',
    'jlk' : 'jelek',
    'smp' : 'sampai',
    'ane' : 'saya',
    'gw' : 'saya',
    'gua' : 'saya',
    'dah' : 'sudah',
    "jelek.": "jelek",
    "dioper": "dioper",
    "lntai3": "lantai 3",
    "1lagi": "1 lagi",
    "lembaga pemerintag": "lembaga pemerintah",
    "payah.": "payah",
    "reformasi": "Reformasi",
    "paket": "paketnya",
    "blum": "belum",
    "dpt": "dapat",
    "pki": "pakai",
    "knp": "kenapa",
    "smpai": "sampai",
    "lmbat": "lambat",
    "dijakarta": "di Jakarta",
    "kerjanya": "kerjanya",
    "cust": "customer",
    "servis": "service",
    "konco": "kawan",
    "ga": "nggak",
    "Emang": "Memang",
    "Gk": "Nggak",
    "Gmn": "Gimana",
    "kl": "kalau",
    "dr": "dari",
    "rmh": "rumah",
    "krj": "kerja",
    "sm": "sama",
    "sdh": "sudah",
    "pd": "pada",
    "jg": "juga",
    "br": "baru",
    "bs": "bisa",
    "hr": "hari",
    "sy": "saya",
    "kt": "kita",
    "ato": "atau",
    "map": "map coklat",
    "coklat": "coklat",
    "blm": "belum",
    "sn": "sana",
    "yg": "yang",
    "payanan": "pelayanan",
    "pertimbanhkan": "pertimbangkan",
    "telfon": "telepon",
    "kagak": "nggak",
    "tlp": "telepon",
    "bodo": "bodoh",
    "bnyak": "banyak",
    "internet": "internetnya",
    "m": "mempermudah",
    "g": "nggak",
    "brp": "berapa",
    "drpd": "daripada",
    "brt": "berat",
    "tlpn": "telepon",
    "alamatnya": "alamat",
    "ngga": "nggak",
    "mentang": "mentang-mentang",
    "sumardi": "Sumardi",
    "tuan": "Tuan",
    "operator e": "operatornya",
    "kedubrak": "Kedodoran",
    "exspress": "express",
    "perna": "pernah",
    "payahhhhh.": "payah",
    "cs": "customer service",
    "tidakdapat": "tidak dapat",
    "Tp": "tapi",
    "ny": "nya",
    "pt.posindonesia": "pt. pos indonesia",
    "krm": "kirim",
    "spt": "seperti",
    "kntr": "kantor",
    "posindonesia": "pos indonesia",
    "nyuruh": "menyuruh",
    "yasudah": "ya sudah",
    "saya bilang": "saya berkata",
    "tasikmlaya": "tasikmalaya",
    "jwb": "jawab",
    "mdl": "modal",
    "tdi": "tadi",
    "seprti": "seperti",
    "tdk": "tidak",
    "telfn": "telepon",
    "no": "nomor",
    "mga": "semoga",
    "negata": "negara",
    "proaea": "proses",
    "ka tor pos": "kantor pos",
    "sms": "saya",
    "mag": "melalui",
    "fb": "facebook",
    "dlm": "dalam",
    "syr": "saya",
    "benerin": "benar",
    "Timur": "timur",
    "indoanesia": "indonesia",
    "Posgiro": "Pos Giro",
    "utang": "utang",
    "ambill": "ambil",
    "bsknyanya": "besoknya",
    "keesookan": "keesokan",
    "ujung2nya": "ujung-ujungnya",
    "ngejawaab": "menjawab",
    "kostumer": "customer",
    "custemer": "customer",
    "ditoellpn": "ditelpon",
    "pengantaran": "pengantaran",
    "hppp": "hape",
    "mgunakan": "menggunakan",
    "kurirnya": "kurir-nya",
    "perlune": "perlu",
"smp": "sampai",
  "telpon": "telepon",
  "knp": "kenapa",
  "bgt": "banget",
  "gak": "tidak",
  "ga": "tidak",
  "blm": "belum",
  "skrng": "sekarang",
  "aja": "saja",
  "tp": "tapi",
  "udh": "sudah",
  "smpai": "sampai",
  "malaes": "males",
  "ngantar": "mengantar",
  "yg": "yang",
  "3hari": "tiga hari",
  "tmpat": "tempat",
  "kesitu": "ke situ",
  "beangnya": "biayanya",
  "skali": "sekali",
  "telp": "telepon",
  "lum": "belum",
  "tgl": "tanggal",
  "dibilang": "dikatakan",
  "cma": "cuma",
  "mw": "mau",
  "jd": "jadi",
  "jgn": "jangan",
  "dr": "dari",
  "sya": "saya",
  "ape": "apa",
  "resek": "rese",
  "ngga": "tidak",
  "klo": "kalau",
  "bakalan": "akan",
  "ky": "kayak",
  "krn": "karena",
  "sth": "setelah",
  "nyampe": "sampai",
  "d": "di",
  "simten": "sistem",
  "d tanggapi": "ditanggapi",
  "2minggu": "dua minggu",
  "2bln": "dua bulan",
  "bikin": "membuat",
  "msih": "masih",
  "sm": "sama",
  "smpe": "sampai",
  "blom": "belum",
  "lah": "saja",
  "sampe": "sampai",
  "dteng": "datang",
  "dn": "dan",
  "dtrima": "diterima",
  "ats": "atas",
  "nma": "nama",
  "nm": "nama",
  "tlfon": "telepon",
  "d angkat": "diangkat",
  "slh": "salah",
  "ato": "atau",
  "kcewa": "kecewa",
  "cek": "periksa",
  "gk": "tidak",
  "anter": "antar",
  "di anter": "diantar",
  "di tahan2": "ditahan-tahan",
  "giliran": "ketika",
  "ngambil": "mengambil",
  "koq": "kok",
  "skarang": "sekarang",
  "seh": "sih",
  "Endonesah": "Indonesia",
  "smpe2": "sampai-sampai",
  "ngendep": "mengendap",
  "pdhl": "padahal",
  "dkt": "dekat",
  "Telfon": "Telepon",
  "lbh": "lebih",
  "di retur": "diretur",
  "di antar2": "diantar-antar",
  "tlpon": "telepon",
  "sprt": "seperti",
  "dri": "dari",
  "sdah": "sudah",
  "sminggu": "seminggu",
  "lbih": "lebih",
  "smpt": "sempat",
  "sayamalah": "saya malah",
  "dkembalikan": "dikembalikan",
  "brng": "barang",
  "brp": "berapa",
  "poa": "pos",
  "ktne": "katanya",
  "brg": "barang",
  "eksoedisi": "ekspedisi",
  "nggak": "tidak",
  "tlg": "tolong",
  "ajh": "aja",
  "mislkan": "misalkan",
  "trmksh": "terima kasih",
  "sampek": "sampai",
  "gabisa": "tidak bisa",
  "tdk": "tidak",
  "WU": "Western Union",
  "gnya": "tidak-nya",
  "dipelayanan": "di pelayanan",
  "dibelakng": "di belakang",
  "g": "tidak",
  "bknya": "bukannya",
  "cs": "customer service",
  "sby": "Surabaya",
  "ktnya": "katanya",
  "pengirimin": "pengiriman",
  "sloooww": "slow",
  "berbelit\"": "berbelit-belit",
  "dtng": "datang",
  "gw": "saya",
  "ampe": "sampai",
  "bsok": "besok",
  "goblokk": "goblok",
  "bangeeet": "banget",
  "yaudah": "ya sudah",
  "kerjanya": "kerja nya",
  "gniat": "tidak niat",
  "kenkantornya": "ke kantornya",
  "gni": "begini",
  "ampas": "buruk",
  "gedor2": "gedor-gedor",
  "klakson2": "klakson-klakson",
  "di anter2": "diantar-antar",
  "bngt": "banget",
  "pket": "paket",
  "tlp": "telepon",
  "dtelpon": "ditelepon",
  "didiemin": "dibiarkan",
  " nya": "nya",
  "kl": "kalau",
  "bsk": "besok",
  "di tlp": "ditelepon",
  "sbnrnya": "sebenarnya",
  "cust": "customer",
  "custemer": "customer",
  "brang": "barang",
  "lmpung": "lampung",
  "dtuju": "dituju",
  "dteruskan": "diteruskan",
  "pk": "pakai",
  "mudik": "pulang kampung",
  "kerumah": "ke rumah",
  "maketin": "mengirimkan",
  "ngapalin": "menghapal",
  "trs": "terus",
  "ny": "nya",
  "dikirimnya": "dikirim",
  "kmrin": "kemarin",
  "kmrn": "kemarin",
  "gausah": "tidak usah",
  "gaakan": "tidak akan",
  "pake": "pakai",
  "kenk": "ke",
  "bru": "baru",
  "byaar": "bayar",
  "brgnya": "barangnya",
  "bsa": "bisa",
  "jng": "jangan",
  "bnyak": "banyak",
  "sy": "saya",
  "dt": "datang",
  "res": "resi",
  "gmna": "bagaimana",
  "bkin": "bikin",
  "mkn": "makan",
  "bnr2": "benar-benar",
  "pliz": "please",
  "tpi": "tapi",
  "kapok": "jerah",
  "ngendon": "mengendap",
  "cus": "customer",
  "mnding": "mending",
  "dgn": "dengan",
  "kirimnya": "dikirim",
  "neng": "adik",
  "cepet": "cepat",
  "dkirim": "dikirim",
  "kirimny": "pengirimannya",
  "mlah": "malah",
  "kiriman": "paket",
  "sb": "sudah",
  "tnggl": "tanggal",
  "trnyata": "ternyata",
  "bln": "bulan",
  "dktr": "dekat",
  "sni": "sini",
  "km": "kamu",
  "lo": "kamu",
  "smua": "semua",
  "kmna": "ke mana",
  "tln": "tolong",
  "dkirim2": "dikirim-kirim",
  "kjra": "kerja",
  "sbnr": "sebenar",
  "masi": "masih",
  "btul": "betul",
  "mlm": "malam",
  "kmren": "kemarin",
  "dtruskan": "diteruskan",
  "yng": "yang",
  "pakrt": "paket",
  "mnt": "minta",
  "pelayananx": "pelayanannya",
  "barng": "barang",
  "skr": "sekarang",
  "jg": "juga",
  "ktanya": "katanya",
  "dhlm": "dalam",
  "komplen": "komplain",
  "negri": "negeri",
  "nmr": "nomor",
  "na": "nya",
  "pmbyrn": "pembayaran",
  "dkasih": "diberikan",
  "antebin": "diabaikan",
  "kga": "tidak",
  "pryanan": "pelayanan",
  "tidk": "tidak",
  "ccd": "cacat",
  "ngentddd": "ngentot",
  "dkembalikn": "dikembalikan",
  "trlebih": "terlebih",
  "d-sms": "di-sms",
  "telfon": "telepon",
  "nyampenny": "sampainya",
  "gue": "saya",
"aj": "saja",
  "didiemin": "dibiarkan",
  "gue": "saya",
  "bngt": "banget",
  "klo": "kalau",
  "kmana": "ke mana",
  "udah": "sudah",
  "mao": "mau",
  "gimana": "bagaimana",
  "dibilang": "dikatakan",
  "kpn": "kapan",
  "nih": "ini",
  "gw": "saya",
  "ngapaen": "ngapain",
  "kalaw": "kalau",
  "kg": "tidak",
  "dksh": "dikasih",
  "no": "nomor",
  "lembr": "lembar",
  "cuman": "hanya",
  "pdhl": "padahal",
  "bkin": "bikin",
  "ampe": "sampai",
  "pmbyrn": "pembayaran",
  "si": "sih",
  "bgt": "banget",
  "dkasi": "diberi",
  "trus": "terus",
  "loketnya": "loketnya",
  "yng": "yang",
  "ga": "tidak",
  "gk": "tidak",
  "g": "tidak",
  "gt": "tidak",
  "ni": "ini",
  "ktr": "kantor",
  "resi": "resi",
  "blm": "belum",
  "sampe": "sampai",
  "udh": "sudah",
  "pdhal": "padahal",
  "dr": "dari",
  "bakalan": "akan",
  "batalin": "batalkan",
  "telpon": "telepon",
  "ditelpon": "ditelepon",
  "diteleponin": "ditelepon",
  "parahhh": "parah",
  "bangat": "banget",
  "dtelponin": "ditelepon",
  "blum": "belum",
  "smpe": "sampai",
  "swtiap": "setiap",
  "hadeuhh": "hadeuh",
  "kevewa": "kecewa",
  "tpi": "tapi",
  "jwab": "jawab",
  "astaghfirullah": "astaghfirullah",
  "bgst": "bangsat",
  "dijawab": "dijawab",
  "gaada": "tidak ada",
  "masak": "masa",
  "alesannya": "alasannya",
  "gak": "tidak",
  "gimana deh": "bagaimana",
  "haha": "ha-ha",
  "coi": "coy",
  "digeletakin": "digeletakkan",
  "jutek-jutek": "jutek",
  "ambyarr": "ambyar",
  "yaampun": "ya ampun",
  "bos": "bos",
  "sma": "sama",
  "aza": "saja",
  "bsa": "bisa",
  "kerja lelet": "kerjanya lambat",
  "ngga": "tidak",
  "ga bagus": "tidak bagus",
  "gak sampai2": "tidak sampai-sampai",
  "pwt": "purwokerto",
  "pbg": "pekalongan",
  "mksudnya": "maksudnya",
  "mbok": "sebaiknya",
  "trs": "terus",
  "smua": "semua",
  "makanya": "maka dari itu",
  "ja": "saja",
  "brat": "banget",
  "amit amit": "amit-amit",
  "sani": "sana",
  "di tlp": "ditelepon",
  "loh": "lho",
  "kapokk": "kapok",
  "lelet": "lambat",
  "susah dihubungi": "sulit dihubungi",
  "enggak": "tidak",
  "engk": "tidak",
  "someday": "nanti",
  "lha": "lah",
  "ga nyampe": "tidak sampai",
  "nyampenya": "sampainya",
  "paketan": "paket",
  "alamatnya": "alamat",
  "alamatnya ga lengkap": "alamat tidak lengkap",
  "pelayananya": "pelayanannya",
  "alamat ga lengkap": "alamat tidak lengkap",
  "alamat gak lengkap": "alamat tidak lengkap",
  "pelayanannya ga jelas": "pelayanannya tidak jelas",
  "pelayanan buruk": "pelayanannya buruk",
  "parahhhh": "parah",
  "pelayanan jelek": "pelayanannya buruk",
  "tai": "buruk",
  "tai kucing": "sangat buruk",
  "mbanya": "mbaknya",
  "ga ngenakin": "tidak mengenakkan",
  "kaya": "seperti",
  "custumer": "customer",
  "kaya driver2": "seperti para driver",
  "nulis nama": "menulis nama",
  "parah banget": "sangat parah",
  "makin ngawur": "semakin kacau",
  "tdk": "tidak",
  "ga ada": "tidak ada",
  "pelayanannya buruk": "pelayanannya buruk",
  "lbh": "lebih",
  "mandi keringet": "berkeringat",
  "pelayanan nya": "pelayanannya",
  "ngirim": "mengirim",
  "gini": "begini",
  "gak mau": "tidak mau",
  "loket hantu": "loket kosong",
  "pake": "pakai",
  "pelayanan prima": "layanan prima",
  "kapan": "sejak kapan",
  "nyantol": "terhenti",
  "berhari²": "berhari-hari",
    "yng": "yang",
    "udh": "sudah",
    "ga": "tidak",
    "g": "tidak",
    "jg": "juga",
    "smpe": "sampai",
    "smp": "sampai",
    "td": "tadi",
    "tlp": "telepon",
    "ditlp": "ditelepon",
    "klo": "kalau",
    "pki": "pakai",
    "dr": "dari",
    "smpai": "sampai",
    "aja": "saja",
    "jd": "jadi",
    "bgt": "banget",
    "bgtu": "begitu",
    "dpt": "dapat",
    "kmren": "kemarin",
    "blm": "belum",
    "dkrim": "dikirim",
    "dhl": "padahal",
    "dgn": "dengan",
    "tp": "tapi",
    "tdk": "tidak",
    "lgsg": "langsung",
    "pdhl": "padahal",
    "nyampe": "sampai",
    "pda": "pada",
    "sy": "saya",
    "bkin": "bikin",
    "trs": "terus",
    "smrg": "semarang",
    "mlng": "malang",
    "sm": "sama",
    "ny": "nya",
    "bln": "bulan",
    "gmna": "bagaimana",
    "cuma": "hanya",
    "nggak": "tidak",
    "grgr": "gara-gara",
    "dhhbungi": "dihubungi",
    "gk": "tidak",
    "sma": "sama",
    "kaga": "tidak",
    "perna": "pernah",
    "pelayananya": "pelayanannya",
    "nmr": "nomor",
    "bbrp": "beberapa",
    "bkn": "bukan",
    "pn": "pun",
    "materii": "materai",
    "cuiuui": "cukup",
    "mater": "maret",
    "trma": "terima",
    "bknya": "bukannya",
    "truss": "terus",
    "anu": "itu",
    "ono": "itu",
    "tau": "tahu",
    "lho": "loh",
    "mpospay": "pospay",
    "trimakasih": "terima kasih",
    "bnyk": "banyak",
    "lagi2": "lagi-lagi",
    "nggk": "tidak",
    "sampek": "sampai",
    "ne": "ini",
    "yg": "yang",
    "ki": "ini",
    "opo": "apa",
    "taukah": "tahu kah",
    "lgi": "lagi",
    "lg": "lagi",
    "kaya": "seperti",
    "mls": "malas",
    "dateng2": "datang-datang",
    "kurirny": "kurirnya",
    "tpi": "tapi",
    "seenggaknya": "setidaknya",
    "dong": "lah",
    "kuyup": "basah",
    "cs": "customer service",
    "info": "informasi",
    "staff": "staf",
    "pasien": "kesabaran",
    "mengerikan": "buruk",
    "jga": "juga",
    "dh": "sudah",
    "lemot": "lambat",
    "gw": "saya",
    "sampe2": "sampai-sampai",
    "blom": "belum",
    "d": "di",
    "sampe": "sampai",
    "pdahal": "padahal",
    "hadeuhhh": "aduh",
    "padhl": "padahal",
    "aturankan": "aturannya",
    "gimana": "bagaimana",
    "nanyain": "menanyakan",
    "gaada": "tidak ada",
    "menau": "mengetahui",
    "belom": "belum",
    "nanya": "bertanya",
    "donk": "dong",
    "sibuk": "sedang sibuk",
    "ekspedisi": "jasa kirim",
    "costumer": "customer",
    "songong": "sombong",
    "madih": "masih",
    "ngga": "tidak",
    "knpa": "kenapa",
    "tlpn": "telepon",
    "lha": "lah",
    "asw": "anjing",
    "oy": "hei",
    "kirim": "mengirim",
    "ngecek": "memeriksa",
    "blagak": "berlagak",
    "doank": "saja",
    "kucuk": "berjalan pelan",
    "dah": "sudah",
    "anjir": "waduh",
    "lhh": "lah",
    "gx": "tidak",
    "pak": "bapak",
    "bu": "ibu",
    "pake": "pakai",
    "nexday": "next day",
    "perbaikilh": "perbaikilah",
    "jngn": "jangan",
    "kecewakn": "mengecewakan",
    "dimna": "di mana",
    "tsb": "tersebut",
    "ngepos": "mengirim",
    "mba2": "mbak-mbak",
    "bapak2": "bapak-bapak",
    "amburadul": "kacau",
    "custumor": "customer",
    "trening": "training",
    "keblakang": "ke belakang",
    "sequrity": "security",
    "myenangkann": "menyenangkan",
    "pek": "paket",
    "aoa": "apa",
    "buble": "bubble",
    "ekpresi": "ekspresi",
    "capeknya": "kelelahannya",
    "malih": "lagi",
    "noll": "nol",
    "dikit": "sedikit"
}

In [4]:
list_sentence_train = []
for sentence in data['review_text_cleaned'] :
    cleaned_sentence = [words_dict[word] if word in list(words_dict.keys()) else word for word in sentence.split()]
    list_sentence_train.append(' '.join(cleaned_sentence))
data['review_text_cleaned'] = list_sentence_train

In [5]:
factory = StopWordRemoverFactory()
stop_words = factory.get_stop_words()
keep_stopwords = ['tidak', 'belum', 'nggak', 'ok','oke', 'kurang']
fix_stopwords = [word for word in stop_words if word not in keep_stopwords]

In [6]:
data['review_text_cleaned'] = data['review_text_cleaned'].apply(
    lambda text: ' '.join([word for word in text.split() if word not in fix_stopwords])
)
data.head(10)

,Table Name,name,Rating,Review Date,review_text,review_text_cleaned
4,kcu_jktcentrum,Irvan Aditya,1.0,2015-05-01,Di Telpon dari Jam setengah 10 pagi sampai jam...,telepon jam setengah 10 pagi jam setengah 2 si...
5,kcu_jktoceania,Senja Anastasja,1.0,2015-05-01,Tracking Number: CW748870756US\nImage of a pro...,tracking number cw748870756us image of a progr...
6,kcu_bandung,vincent sumarga,1.0,2016-05-01,"Pengiriman ""kilat"" sudah hari k 3 blm dtg. Tel...",pengiriman kilat hari 3 belum datang telepon k...
8,kcu_bekasi,Risky Komara,1.0,2016-05-01,"maaf, Sy cb telfon Kok tdk bs y??\nmau Tanya ...",maaf coba telepon kok tidak mau tanya pengirim...
9,kcu_jktcentrum,mayvits vita,1.0,2016-05-01,Paket saya dr jakpus ke tangerang sudah smingg...,paketnya jakpus tangerang seminggu belum dteng...
10,kcu_jktcentrum,Riko Fajar,1.0,2016-05-01,Mengecewakan. Telpon ga pernah diangkat,mengecewakan telepon tidak pernah diangkat
11,kcu_jktcentrum,Siti Komariah,1.0,2016-05-01,Cara ambil kiriman darri luar negeri bagaimana...,cara ambil paket darri luar negeri bagaimana b...
12,kcu_jktcentrum,Suprianto,1.0,2016-05-01,nomer telepon ga diangkat sama sekali sudah 5 ...,nomer telepon tidak diangkat sama sekali 5 har...
13,kcu_jktcentrum,Vebbe Valerie,1.0,2016-05-01,untuk menghubungi kantor pos pusat di pasar ba...,menghubungi kantor pos pusat pasar baru susah ...
14,kcu_jktflora,kenny pratama,1.0,2016-05-01,"parah , gak ada yang angkat telpon, kacau bang...",parah tidak angkat telepon kacau sekali sdm ny...


In [7]:
# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# stem
stem = []

for index, row in data.iterrows():
    try:
        sentence = row['review_text_cleaned']
        result = stemmer.stem(sentence)
    except Exception as e:
        result = 'error'
    stem.append(result)

data['review_text_cleaned'] = stem
data.head(10)

,Table Name,name,Rating,Review Date,review_text,review_text_cleaned
4,kcu_jktcentrum,Irvan Aditya,1.0,2015-05-01,Di Telpon dari Jam setengah 10 pagi sampai jam...,telepon jam tengah 10 pagi jam tengah 2 siang ...
5,kcu_jktoceania,Senja Anastasja,1.0,2015-05-01,Tracking Number: CW748870756US\nImage of a pro...,tracking number cw748870756us image of a progr...
6,kcu_bandung,vincent sumarga,1.0,2016-05-01,"Pengiriman ""kilat"" sudah hari k 3 blm dtg. Tel...",kirim kilat hari 3 belum datang telepon kantor...
8,kcu_bekasi,Risky Komara,1.0,2016-05-01,"maaf, Sy cb telfon Kok tdk bs y??\nmau Tanya ...",maaf coba telepon kok tidak mau tanya kirim ki...
9,kcu_jktcentrum,mayvits vita,1.0,2016-05-01,Paket saya dr jakpus ke tangerang sudah smingg...,paket jakpus tangerang minggu belum dteng2 tln...
10,kcu_jktcentrum,Riko Fajar,1.0,2016-05-01,Mengecewakan. Telpon ga pernah diangkat,kecewa telepon tidak pernah angkat
11,kcu_jktcentrum,Siti Komariah,1.0,2016-05-01,Cara ambil kiriman darri luar negeri bagaimana...,cara ambil paket darri luar negeri bagaimana b...
12,kcu_jktcentrum,Suprianto,1.0,2016-05-01,nomer telepon ga diangkat sama sekali sudah 5 ...,nomer telepon tidak angkat sama sekali 5 hari ...
13,kcu_jktcentrum,Vebbe Valerie,1.0,2016-05-01,untuk menghubungi kantor pos pusat di pasar ba...,hubung kantor pos pusat pasar baru susah sekal...
14,kcu_jktflora,kenny pratama,1.0,2016-05-01,"parah , gak ada yang angkat telpon, kacau bang...",parah tidak angkat telepon kacau sekali sdm ny...


In [8]:
kata = {
    "buruk", "lambat", "jelek", "tidak", "mengecewakan", "angkat",'telat','lelet', 'lemot','kecewa','jutek', 'cuek',
    "lama", "lelet", "cepat", "ramah", "baik",'puas','mantab', "memuaskan", "terbaik", "kurang",'bantu','jawab'
}

hasil_konteks = []

for text in data['review_text_cleaned']:
    words = text.split()
    konteks = []

    for i in range(len(words)):
        current = words[i]

        # Cek unigram
        if current in kata:
            # Ambil sebelumnya (jika ada)
            prev = words[i - 1] if i - 1 >= 0 else ''
            # Ambil sesudahnya (jika ada)
            next_ = words[i + 1] if i + 1 < len(words) else ''
            konteks.append(' '.join([prev, current]).strip())
            konteks.append(' '.join([current, next_]).strip())
            konteks.append(' '.join([prev,current, next_]).strip())

        # Cek bigram (tidak ramah, dll)
        if i > 0:
            bigram = f"{words[i-1]} {words[i]}"
            if bigram in kata:
                # Ambil kata sebelum bigram dan sesudahnya
                prev = words[i - 2] if i - 2 >= 0 else ''
                next_ = words[i + 1] if i + 1 < len(words) else ''
                konteks.append(' '.join([prev, bigram]).strip())
                konteks.append(' '.join([bigram, next_]).strip())
                konteks.append(' '.join([prev, bigram, next_]).strip())

    hasil_konteks.append(konteks)

# Masukkan hasil ke kolom baru
data['kata_konteks'] = hasil_konteks

In [9]:
data.head(50)

,Table Name,name,Rating,Review Date,review_text,review_text_cleaned,kata_konteks
4,kcu_jktcentrum,Irvan Aditya,1.0,2015-05-01,Di Telpon dari Jam setengah 10 pagi sampai jam...,telepon jam tengah 10 pagi jam tengah 2 siang ...,"[sekali tidak, tidak angkat, sekali tidak angk..."
5,kcu_jktoceania,Senja Anastasja,1.0,2015-05-01,Tracking Number: CW748870756US\nImage of a pro...,tracking number cw748870756us image of a progr...,"[paket tidak, tidak antar, paket tidak antar]"
6,kcu_bandung,vincent sumarga,1.0,2016-05-01,"Pengiriman ""kilat"" sudah hari k 3 blm dtg. Tel...",kirim kilat hari 3 belum datang telepon kantor...,"[sekali angkat, angkat, sekali angkat]"
8,kcu_bekasi,Risky Komara,1.0,2016-05-01,"maaf, Sy cb telfon Kok tdk bs y??\nmau Tanya ...",maaf coba telepon kok tidak mau tanya kirim ki...,"[kok tidak, tidak mau, kok tidak mau]"
9,kcu_jktcentrum,mayvits vita,1.0,2016-05-01,Paket saya dr jakpus ke tangerang sudah smingg...,paket jakpus tangerang minggu belum dteng2 tln...,"[nya tidak, tidak dteng2, nya tidak dteng2]"
10,kcu_jktcentrum,Riko Fajar,1.0,2016-05-01,Mengecewakan. Telpon ga pernah diangkat,kecewa telepon tidak pernah angkat,"[kecewa, kecewa telepon, kecewa telepon, telep..."
11,kcu_jktcentrum,Siti Komariah,1.0,2016-05-01,Cara ambil kiriman darri luar negeri bagaimana...,cara ambil paket darri luar negeri bagaimana b...,[]
12,kcu_jktcentrum,Suprianto,1.0,2016-05-01,nomer telepon ga diangkat sama sekali sudah 5 ...,nomer telepon tidak angkat sama sekali 5 hari ...,"[telepon tidak, tidak angkat, telepon tidak an..."
13,kcu_jktcentrum,Vebbe Valerie,1.0,2016-05-01,untuk menghubungi kantor pos pusat di pasar ba...,hubung kantor pos pusat pasar baru susah sekal...,"[sekali angkat, angkat tidak, sekali angkat ti..."
14,kcu_jktflora,kenny pratama,1.0,2016-05-01,"parah , gak ada yang angkat telpon, kacau bang...",parah tidak angkat telepon kacau sekali sdm ny...,"[parah tidak, tidak angkat, parah tidak angkat..."


In [10]:
from collections import Counter

# Gabungkan semua kata_konteks
all_konteks = [item for sublist in data['kata_konteks'] for item in sublist]

# Pisahkan berdasarkan jumlah kata
konteks_2_kata = [k for k in all_konteks if len(k.split()) == 2]
konteks_3_kata = [k for k in all_konteks if len(k.split()) == 3]

# Hitung frekuensi masing-masing
counter_2 = Counter(konteks_2_kata)
counter_3 = Counter(konteks_3_kata)

# Ubah jadi DataFrame dan urutkan
df_2_kata = pd.DataFrame(counter_2.items(), columns=['kata_konteks', 'frekuensi'])
df_2_kata = df_2_kata.sort_values(by='frekuensi', ascending=False).reset_index(drop=True)

df_3_kata = pd.DataFrame(counter_3.items(), columns=['kata_konteks', 'frekuensi'])
df_3_kata = df_3_kata.sort_values(by='frekuensi', ascending=False).reset_index(drop=True)

In [11]:
df_2_kata

,kata_konteks,frekuensi
0,tidak angkat,209
1,layan ramah,190
2,layan baik,148
3,layan cepat,140
4,tidak ramah,129
...,...,...
3992,cepat pertama,1
3993,ramah mba,1
3994,lama kunjung,1
3995,ramah senyum,1


In [12]:
df_3_kata

,kata_konteks,frekuensi
0,telepon tidak pernah,32
1,telepon tidak angkat,29
2,layan ramah cepat,25
3,layan cepat ramah,20
4,layan baik ramah,13
...,...,...
4328,pemberitahuan tidak bunyi,1
4329,angkat tidak dijawb,1
4330,sekali angkat tidak,1
4331,nya tidak dteng2,1


In [4]:
import pandas as pd
import re
import string
from transformers import pipeline
from tqdm import tqdm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

In [6]:
ayameRushia = "ayameRushia/bert-base-indonesian-1.5G-sentiment-analysis-smsa"
ayameRushia_model = pipeline(
    "sentiment-analysis",
    model=ayameRushia,
    tokenizer=ayameRushia,
    truncation=True,
     max_length=512
)

Device set to use cpu


In [7]:
Aardiiiiy = "Aardiiiiy/indobertweet-base-Indonesian-sentiment-analysis"
Aardiiiiy_model = pipeline(
    "sentiment-analysis",
    model=Aardiiiiy,
    tokenizer=Aardiiiiy,
    truncation=True,
     max_length=512
)

c:\Users\POS\AppData\Local\Programs\Python\Python313\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\POS\.cache\huggingface\hub\models--Aardiiiiy--indobertweet-base-Indonesian-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu
